In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\gusti\Notebooks\urgencias\AtencionesUrgencia2022.csv", encoding="latin1", sep=";")

In [3]:
df["Año"] = df["fecha"].apply(lambda x: str(x)[30:35])

In [4]:
del df["MENOR_A_1"]
del df["Column7"]
del df["__14"]
del df["_5_64"]
del df["_5_MAS"]

In [5]:
df.columns

Index(['idestablecimiento', 'nestablecimiento', 'Idcausa', 'glosacausa',
       'TOTAL', 'fecha', 'semana', 'GLOSATIPOESTABLECIMIENTO',
       'GLOSATIPOATENCION', 'GlosaTipoCampana', 'Año'],
      dtype='object')

In [6]:
df2 = df.groupby(by=['idestablecimiento', 'nestablecimiento', 'Idcausa', 'glosacausa',
                     'semana', 'GLOSATIPOESTABLECIMIENTO',
                     'GLOSATIPOATENCION', 'GlosaTipoCampana', 'Año']).sum()

In [7]:
df2 = df2.reset_index()
#df2.to_excel("PruebaUrgencias2022.xlsx", index = False)

In [8]:
dfRef = pd.read_excel(r"C:\Users\gusti\Notebooks\RefUrgencias.xlsx")

In [9]:
dfRef2 = pd.read_excel(r"C:\Users\gusti\Notebooks\RefRegionUrgencias.xlsx")

In [10]:
dfExcluidos = pd.read_excel(r"C:\Users\gusti\Notebooks\RefUrgencias.xlsx", sheet_name = "Excluidos")

In [11]:
df3 = df2
df2_1 = df2[df2["semana"]==df2["semana"].unique()[0]]
df2_1[df2["semana"].unique()[0]] = df2_1["TOTAL"]
del df2_1["TOTAL"]
del df2_1["semana"]
for i in df2["semana"].unique()[1:]:
    dfAux = df2[df2["semana"] == i]
    dfAux[i] = dfAux["TOTAL"]
    del dfAux["semana"]
    del dfAux["TOTAL"]
    df2_1 = df2_1.merge(dfAux, left_on=['idestablecimiento', 'nestablecimiento', 'Idcausa', 'glosacausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
       'GlosaTipoCampana', 'Año'], right_on= ['idestablecimiento', 'nestablecimiento', 'Idcausa', 'glosacausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
       'GlosaTipoCampana', 'Año'])

<ipython-input-11-fb27c1066df6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_1[df2["semana"].unique()[0]] = df2_1["TOTAL"]
<ipython-input-11-fb27c1066df6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfAux[i] = dfAux["TOTAL"]
<ipython-input-11-fb27c1066df6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [14]:
dfFix2 = df2_1[['glosacausa',1,2,3,4,5,6,7,8,9,10,11,12,13,'Año','GLOSATIPOESTABLECIMIENTO'
                ,'nestablecimiento','GLOSATIPOATENCION']]

In [15]:
dfFix3 = dfFix2.rename(columns={'Año':'Fecha','GLOSATIPOESTABLECIMIENTO':'Tipo Establecimiento',
                                'nestablecimiento':'Nombre establecimiento','GLOSATIPOATENCION':'Tipo'})

In [16]:
excluidos = list(dfExcluidos["Excluidos"])

In [17]:
dfFix4 = dfFix3[dfFix3["glosacausa"].apply(lambda x: x not in excluidos)]

In [18]:
dfMerge = dfFix4.merge(dfRef, left_on= "glosacausa", right_on= "Urgencia", how= "inner")

In [19]:
dfMergeFix = dfMerge[['Urgencia',1,2,3,4,5,6,7,8,9,10,11,12,13,'Fecha'
                      ,'Tipo Establecimiento','Nombre establecimiento','Tipo','Causas']]

In [20]:
dfFinal = dfMergeFix.merge(dfRef2, left_on= "Nombre establecimiento", right_on= "Nombre establecimiento", how= "inner")

In [21]:
dfFinal2 = dfFinal[['Urgencia',1,2,3,4,5,6,7,8,9,10,11,12,13,'Fecha','Region'
                    ,'Tipo Establecimiento','Nombre establecimiento','Tipo','Causas']]

In [22]:
dfFinal2.to_excel("Urgencias2022.xlsx", index = False)